In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
PROJECT_ID = "claimwise-ai-469715"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [ ]:
!bq mk --connection --location=us \
    --connection_type=CLOUD_RESOURCE test_connection

BigQuery error in mk operation: Already Exists: Connection
projects/477012212019/locations/us/connections/test_connection


In [ ]:
SERVICE_ACCT = !bq show --format=prettyjson --connection us.test_connection | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

bqcx-477012212019-1fi4@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [ ]:
import time

!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/aiplatform.user'

# wait 60 seconds, give IAM updates time to propagate, otherwise, following cells will fail
time.sleep(60)

Updated IAM policy for project [claimwise-ai-469715].
Updated IAM policy for project [claimwise-ai-469715].


In [ ]:
%%bigquery cars --project {PROJECT_ID}
SELECT
  policy_number,
  auto_make,
  auto_model,
  AI.GENERATE(
    # 'Summarize this car: make=' || CAST(auto_make AS STRING) ||
    #  ', model=' || CAST(auto_model AS STRING) ||
    #  ', incident_type=' || CAST(incident_type AS STRING) ||
    #  ', total claim=' || total_claim_amount ||
    #  ', incident location=' || incident_location || ',' || incident_city,
    'Summarize this car based on ' || auto_make || auto_model ||
    incident_type || total_claim_amount || incident_location || incident_city,
    connection_id => 'us.test_connection',
    endpoint => 'gemini-2.5-flash'
  ).result AS car_summary
FROM `claimwise-ai-469715.claimwise.insurance_claims`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
import pandas as pd

# remove truncation for long text fields
pd.set_option("display.max_colwidth", None)

In [ ]:
cars

,policy_number,auto_make,auto_model,car_summary
0,427199,BMW,3 Series,"A BMW 3 Series was involved in a multi-vehicle collision at 457002469 Francis Lane, Northbend."
1,444035,Saab,92x,The car in question is a **Saab 9-2X** that was involved in a **multi-vehicle collision** on **5th Ave in Hillsdale**. The incident is referenced by the number **427003397**.
2,633375,BMW,3 Series,This is a **BMW 3 Series** car that was involved in a **multi-vehicle collision**. The incident occurred on **Rock Drive** in **Columbus** and is associated with the reference number **796807630**.
3,531640,BMW,3 Series,"Based on the information provided, the car is a **BMW 3 Series** that was involved in a **multi-vehicle collision** on **July 28, 2016**, at **28 Best Drive in Arlington**."
4,497347,Saab,92x,"Based on the provided information, the car is a:\n\n* **Make & Model:** **Saab 9-2x**\n* **Context:** It was involved in a **multi-vehicle collision** on **Lincoln Hwy in Columbus**. The number ""543409720"" likely refers to an incident or case number."
5,507545,Saab,92x,"Based on the provided string, the car is identified as a **Saab 9-2X**.\n\nThe string indicates that this specific Saab 9-2X was involved in a **multi-vehicle collision**. The incident appears to have occurred near **Flute Ridge in Columbus**, and ""544506859"" likely serves as an **incident or case number** related to the event.\n\nThe string provides no further details about the car's condition, year, color, or specific features beyond its make and model and its involvement in the collision."
6,469874,Saab,92x,"The car in question is a **Saab 9-2X**.\n\nIt was involved in a **collision** in the **Northbend** area. The description provides conflicting information regarding the nature of the collision, stating it was both a **""Multi-vehicle Collision""** and a **""Solo Drive""**. This could suggest the Saab 9-2X's specific involvement was a solo incident within a broader multi-vehicle event, or that the details are inconsistent."
7,336614,BMW,3 Series,"This refers to a **BMW 3 Series** vehicle involved in a **multi-vehicle collision** at **5th Ave, Arlington**. The incident is identified by the number **297906092**."
8,373731,BMW,3 Series,"Based on the information provided, the car is a **BMW 3 Series** that was involved in a **multi-vehicle collision** on **Pine Ave** in **Springfield**. The incident is associated with the reference number **815004567**."
9,599174,Saab,92x,"Based on the information provided:\n\nThe car in question is a **Saab 9-2X**.\n\nHere's a summary of the car, followed by what the rest of the text refers to:\n\n**Saab 9-2X Car Summary:**\nThe Saab 9-2X is a compact executive car that was sold by Saab for the 2005 and 2006 model years. It is most notable for being a rebadged and mildly restyled version of the Subaru Impreza (specifically the second-generation ""bug-eye"" and ""blob-eye"" models), produced during General Motors' ownership of both Saab and a stake in Subaru's parent company, Fuji Heavy Industries.\n\n* **""Saabaru"":** It quickly earned the nickname ""Saabaru"" due to its Subaru underpinnings.\n* **Mechanicals:** It largely shared its mechanical components, including the all-wheel-drive system and boxer engines, with the Subaru Impreza and WRX. Two main engine options were available: a naturally aspirated 2.5L flat-four (from the Impreza 2.5 RS) and a turbocharged 2.0L or 2.5L flat-four (from the WRX).\n* **Design Differences:** Saab made minor styling changes to the exterior (mostly the front fascia, grille, and some rear lighting) and interior (different seats, dashboard trim) to align it more with the Saab brand aesthetic.\n* **Purpose:** It was intended to give Saab a competitor in the compact performance market and a smaller, entry-level model to boost sales in North America.\n\nThe additional information in your prompt (""Multi-vehicle Collision 480702014 Rock Ave Arlington"") indicates that a Saab 9-2X was involved in a **multi-vehicl

In [ ]:
%%bigquery df --project {PROJECT_ID}

SELECT
  *,
  CAST(
    AI.GENERATE(
      'Find the most similar case with this information: auto_make=Ford, auto_model=Fusion, auto_year=1998, collision_type=Rear Collision' || '. Respond a number from 0 to 100 representing similarity percentage.',
      connection_id => 'us.test_connection',
      endpoint => 'gemini-2.5-flash'
    ).result AS FLOAT64
  ) AS similarity_score
FROM `claimwise-ai-469715.claimwise.insurance_claims`
ORDER BY similarity_score DESC
LIMIT 10;

Executing query with job ID: 2db8968d-3ee8-4765-b9b4-58aeb50e5850
Query executing: 194.30s


ERROR:
 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/claimwise-ai-469715/queries/2db8968d-3ee8-4765-b9b4-58aeb50e5850?maxResults=0&location=US&prettyPrint=false: Bad double value: The provided information `auto_m...

Location: US
Job ID: 2db8968d-3ee8-4765-b9b4-58aeb50e5850



In [ ]:
insurance_claims = pd.read_csv('insurance_claims.csv')
insurance_claims.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


# 3 code cells below are used only for creating GCS bucket from google drive media folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gsutil mb -l us-east1 gs://vehicle_media

Creating gs://insurance-media/...


In [ ]:
!gsutil -m cp /content/drive/MyDrive/insurance_media/* gs://vehicle_media/

Copying file:///content/drive/MyDrive/insurance_media/car-crash-1.jpeg [Content-Type=image/jpeg]...
Copying file:///content/drive/MyDrive/insurance_media/car-crash2.jpg [Content-Type=image/jpeg]...
Copying file:///content/drive/MyDrive/insurance_media/car-crash.jpeg [Content-Type=image/jpeg]...
Copying file:///content/drive/MyDrive/insurance_media/car-crash-5.jpg [Content-Type=image/jpeg]...
Copying file:///content/drive/MyDrive/insurance_media/67fdf83a892ab.image.jpg [Content-Type=image/jpeg]...
\ [5/5 files][484.0 KiB/484.0 KiB] 100% Done                                    
Operation completed over 5 objects/484.0 KiB.                                    


In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE EXTERNAL TABLE `claimwise-ai-469715.claimwise.vehicle_medias`
WITH CONNECTION `us.test_connection`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['gs://vehicle_media/*']
);

Query is running:   0%|          |

""


In [ ]:
!gsutil iam ch serviceAccount:bqcx-477012212019-1fi4@gcp-sa-bigquery-condel.iam.gserviceaccount.com:roles/storage.objectViewer gs://vehicle_media

In [ ]:
%%bigquery df --project {PROJECT_ID}

SELECT * FROM `claimwise-ai-469715.claimwise.vehicle_medias`;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,uri,generation,content_type,size,md5_hash,updated,metadata,ref
0,gs://vehicle_media/67fdf83a892ab.image.jpg,1756136877030852,image/jpeg,22239,44583ac7551f548c003051e3b3808c10,2025-08-25 15:47:57.087000+00:00,[],"{'uri': 'gs://vehicle_media/67fdf83a892ab.image.jpg', 'version': '1756136877030852', 'authorizer': 'claimwise-ai-469715.us.test_connection', 'details': '{""gcs_metadata"":{""content_type"":""image/jpeg"",""md5_hash"":""44583ac7551f548c003051e3b3808c10"",""size"":22239,""updated"":1756136877087000}}'}"
1,gs://vehicle_media/car-crash-1.jpeg,1756136877631205,image/jpeg,202572,cdf037d9a46c1e44d90c594cef06c42c,2025-08-25 15:47:57.687000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash-1.jpeg', 'version': '1756136877631205', 'authorizer': 'claimwise-ai-469715.us.test_connection', 'details': '{""gcs_metadata"":{""content_type"":""image/jpeg"",""md5_hash"":""cdf037d9a46c1e44d90c594cef06c42c"",""size"":202572,""updated"":1756136877687000}}'}"
2,gs://vehicle_media/car-crash-5.jpg,1756136877629487,image/jpeg,53572,4f673ed3fefc1e18502a8a006ff1585e,2025-08-25 15:47:57.668000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash-5.jpg', 'version': '1756136877629487', 'authorizer': 'claimwise-ai-469715.us.test_connection', 'details': '{""gcs_metadata"":{""content_type"":""image/jpeg"",""md5_hash"":""4f673ed3fefc1e18502a8a006ff1585e"",""size"":53572,""updated"":1756136877668000}}'}"
3,gs://vehicle_media/car-crash.jpeg,1756136877033490,image/jpeg,10346,4271730b6d59d90b3f448d05ebd26f81,2025-08-25 15:47:57.088000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash.jpeg', 'version': '1756136877033490', 'authorizer': 'claimwise-ai-469715.us.test_connection', 'details': '{""gcs_metadata"":{""content_type"":""image/jpeg"",""md5_hash"":""4271730b6d59d90b3f448d05ebd26f81"",""size"":10346,""updated"":1756136877088000}}'}"
4,gs://vehicle_media/car-crash2.jpg,1756136877031047,image/jpeg,206916,b7372a869815288c4818f51370191e55,2025-08-25 15:47:57.068000+00:00,[],"{'uri': 'gs://vehicle_media/car-crash2.jpg', 'version': '1756136877031047', 'authorizer': 'claimwise-ai-469715.us.test_connection', 'details': '{""gcs_metadata"":{""content_type"":""image/jpeg"",""md5_hash"":""b7372a869815288c4818f51370191e55"",""size"":206916,""updated"":1756136877068000}}'}"


In [ ]:
%%bigquery df --project $PROJECT_ID

SELECT
  uri,
  CAST(
    AI.GENERATE(
      STRUCT(
        'Describe what kind of car crash is shown in this image: ', ref),
      connection_id => 'us.test_connection',
      endpoint => 'gemini-2.5-flash'
    ).result AS STRING
  ) AS crash_description
FROM `claimwise-ai-469715.claimwise.vehicle_medias`;

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,uri,crash_description
0,gs://vehicle_media/car-crash-1.jpeg,"This image depicts a **single-vehicle accident** where a black Chevrolet sedan has **collided with a fixed object**, specifically a sturdy metal pole located on the sidewalk.\n\nHere's a breakdown of the scene:\n\n* **Vehicle Involved:** A black Chevrolet sedan (likely a Cruze, given the body style and grille).\n* **Point of Impact:** The car sustained severe **frontal impact damage**. The hood is significantly crumpled, the front bumper is detached and broken, the grille is destroyed, and components beneath the hood are exposed.\n* **Object Hit:** The car has crashed directly into a thick, rusty brown metal pole on the sidewalk. Debris from the car is scattered around the base of the pole.\n* **Airbag Deployment:** Multiple airbags inside the passenger cabin are clearly deployed (visible through the windows), indicating a substantial force of impact.\n* **Vehicle Position:** The car is partially on the sidewalk, pressed against the pole, and partially on the roadway, near a double yellow line.\n* **Emergency Response:** A white police patrol vehicle with its lights activated is visible in the background, indicating that emergency services are on the scene to manage the aftermath of the crash.\n* **Overall Scene:** The crash appears to be significant, rendering the vehicle undrivable due to the extensive front-end damage and airbag deployment."
1,gs://vehicle_media/car-crash-5.jpg,"The image shows a **two-vehicle car crash**, specifically an **offset rear-end collision**.\n\nHere's a breakdown:\n\n1. **Vehicles Involved:**\n * A light-colored (likely silver or light grey) sedan.\n * A darker-colored (dark grey or blue-grey) sedan.\n\n2. **Type of Impact:**\n * The front of the light-colored car has collided with the rear of the darker-colored car.\n * It's an **offset** impact: the front right side (passenger side) of the silver car struck the rear left side (driver's side) of the darker car.\n\n3. **Damage Sustained:**\n * **Light-colored car:** Shows substantial damage to its front right fender, bumper, and possibly the hood and headlight area, which are crumpled, torn, and pushed inward. An airbag (appears to be the driver's side) is deployed inside the vehicle.\n * **Darker-colored car:** Exhibits significant damage to its rear bumper and trunk area, particularly on the left side, where it is heavily dented and crushed inward.\n\n4. **Setting:** The crash appears to have occurred on an asphalt road or street under bright daylight conditions, as evidenced by the clear shadows."
2,gs://vehicle_media/car-crash2.jpg,"The image shows a **low-speed impact** type of car crash, specifically damaging the **rear right (passenger side) quarter panel and bumper** of a blue car.\n\nHere's a breakdown of the damage, indicating the nature of the crash:\n\n* **Significant Denting:** There's a prominent inward dent on the corner of the bumper and extending onto the quarter panel. This suggests a direct impact point.\n* **Creasing and Cracking:** The plastic bumper material is deeply creased and shows a distinct crack at the deepest point of the indentation, indicating the force was enough to deform and fracture the material.\n* **Scratches and Scuff Marks:** Numerous white scuff marks and black scratches are visible around the dented area. These suggest a scraping motion or contact with a rough surface or another vehicle's paint, rather than just a clean, direct hit. The scuffing indicates friction and movement during the impact.\n* **Panel Separation (minor):** In some views, it appears there might be a slight separation or misalignment of the bumper/fender panel from the adjacent bodywork, which can happen with angled impacts.\n\n**In summary, this appears to be the result of:**\n\n* A **sideswipe collision** with another vehicle or object.\n* The car **backing into a stationary object** like a pole, wall, or another parked car at an angle.\n* A **fender bender** where the cor